In [7]:
# Load YOLOv8n, train it on COCO128 for 3 epochs and predict an image with it
from ultralytics import YOLO
import os
import shutil  # To handle file removing

In [8]:
source = "image_face_test/two_faces/*.jpg"

In [9]:
model = YOLO(
    "yolov8_model/yolov8n-face.pt"
)  # load a pretrained YOLOv8n detection model

In [10]:
if os.path.exists("runs"):
    shutil.rmtree("runs")
results = model(source, save_crop=True)
# results.save_crop('image_face_test/two_faces/cropped')


image 1/2 d:\face-matching-models\image_face_test\two_faces\moad.jpg: 384x640 1 face, 280.7ms
image 2/2 d:\face-matching-models\image_face_test\two_faces\moad_pass (1).jpg: 640x448 1 face, 277.5ms
Speed: 13.3ms preprocess, 279.1ms inference, 15.8ms postprocess per image at shape (1, 3, 640, 448)
Results saved to d:\face-matching-models\runs\detect\predict


### Extract Face Features


#### 1. Get the cropped faces


In [11]:
import glob

In [12]:
target_cropped_faces_dir = r"runs\detect\predict\crops\face"
# This folder was created by YOLO

In [13]:
faces_images = glob.glob(os.path.join(target_cropped_faces_dir, "*.jpg"))

# Loop through the list of image files
for image_file in faces_images:
    print(image_file)

runs\detect\predict\crops\face\moad.jpg
runs\detect\predict\crops\face\moad_pass (1).jpg


#### 2. Extract Features


DeepFace library version 0.0.80 is used to do so.


In [14]:
# !pip install deepface==0.0.80

In [15]:
from deepface.commons import distance as dst
from deepface.DeepFace import represent

##### 2.1 Define Parameters


In [16]:
mode_name = "VGG-Face"
distance_metric = ""
detector_backend = "opencv"
align = True
normalization = "base"

_This is from deepface v0.0.80 source code:_

def findThreshold(model_name, distance_metric):

    base_threshold = {"cosine": 0.40, "euclidean": 0.55, "euclidean_l2": 0.75}

    thresholds = {
        "VGG-Face": {"cosine": 0.40, "euclidean": 0.60, "euclidean_l2": 0.86},
        "Facenet": {"cosine": 0.40, "euclidean": 10, "euclidean_l2": 0.80},
        "Facenet512": {"cosine": 0.30, "euclidean": 23.56, "euclidean_l2": 1.04},
        "ArcFace": {"cosine": 0.68, "euclidean": 4.15, "euclidean_l2": 1.13},
        "Dlib": {"cosine": 0.07, "euclidean": 0.6, "euclidean_l2": 0.4},
        "SFace": {"cosine": 0.593, "euclidean": 10.734, "euclidean_l2": 1.055},
        "OpenFace": {"cosine": 0.10, "euclidean": 0.55, "euclidean_l2": 0.55},
        "DeepFace": {"cosine": 0.23, "euclidean": 64, "euclidean_l2": 0.64},
        "DeepID": {"cosine": 0.015, "euclidean": 45, "euclidean_l2": 0.17},
    }

    threshold = thresholds.get(model_name, base_threshold).get(distance_metric, 0.4)

    return threshold


_This is the docstring of represent function:_

"""
This function represents facial images as vectors. The function uses convolutional neural
networks models to generate vector embeddings.

    Parameters:
            img_path (string): exact image path. Alternatively, numpy array (BGR) or based64
            encoded images could be passed. Source image can have many faces. Then, result will
            be the size of number of faces appearing in the source image.

            model_name (string): VGG-Face, Facenet, Facenet512, OpenFace, DeepFace, DeepID, Dlib,
            ArcFace, SFace

            enforce_detection (boolean): If no face could not be detected in an image, then this
            function will return exception by default. Set this to False not to have this exception.
            This might be convenient for low resolution images.

            detector_backend (string): set face detector backend to opencv, retinaface, mtcnn, ssd,
            dlib, mediapipe or yolov8.

            align (boolean): alignment according to the eye positions.

            normalization (string): normalize the input image before feeding to model

    Returns:
            Represent function returns a list of object with multidimensional vector (embedding).
            The number of dimensions is changing based on the reference model.
            E.g. FaceNet returns 128 dimensional vector; VGG-Face returns 2622 dimensional vector.
    """


##### 2.2 Represent Faces


In [17]:
faces_images = glob.glob(os.path.join(target_cropped_faces_dir, "*.jpg"))

# Loop through the list of image files
for image_file in faces_images:
    face_1_representation = represent(
        img_path=image_file,
        model_name=mode_name,
        enforce_detection=False,
        detector_backend=detector_backend,
        align=align,
        normalization=normalization,
    )

    face_2_representation = represent(
        img_path=image_file,
        model_name=mode_name,
        enforce_detection=False,
        detector_backend=detector_backend,
        align=align,
        normalization=normalization,
    )